In [2]:
import pickle
import requests
from bs4 import BeautifulSoup,Comment

In [ ]:
with open(f"./data/scrap/cards_urls.pickle", "rb") as input_file:
   cards_urls = pickle.load(input_file)

with open(f"./data/scrap/sets.pickle", "rb") as input_file:
   sets_data = pickle.load(input_file)

with open(f"./data/scrap/cards_metadata.pickle", "rb") as input_file:
   cards_metadata = pickle.load(input_file)

with open(f"./data/scrap/cards_collections.pickle", "rb") as input_file:
   sets = pickle.load(input_file)

import json
with open("./faiss_db/pokemon-cards_schema.json") as f:
   final_table = json.load(f)

corresponsance_id_ptcgo_code = {}
for card_set in sets_data:
    corresponsance_id_ptcgo_code[card_set.id] = card_set.ptcgoCode



In [ ]:
#for idx,card in enumerate(cards_metadata):
    #print("index : ",str(idx) , "oldvaluer:" , cards_metadata[idx]['name'] , " new :",final_table[idx]["id"])


In [ ]:
session = requests.Session()



index :  0 oldvaluer: ASR_133_R_EN_LG.png  new : ASR-133
index :  1 oldvaluer: 56_hires.png  new : CG-56
index :  2 oldvaluer: SSH_085_R_EN_LG.png  new : SSH-85
index :  3 oldvaluer: 49_hires.png  new : G2-49
index :  4 oldvaluer: 78_hires.png  new : MA-78
index :  5 oldvaluer: ASR_114_R_EN_LG.png  new : ASR-114
index :  6 oldvaluer: 82_hires.png  new : EM-82
index :  7 oldvaluer: AOR_098_R_EN_LG.png  new : AOR-98
index :  8 oldvaluer: PAR_152_R_EN_LG.png  new : PAR-152
index :  9 oldvaluer: 98_hires.png  new : SW-98
index :  10 oldvaluer: CRZ_128_R_EN_LG.png  new : CRZ-128
index :  11 oldvaluer: 53_hires.png  new : N1-53
index :  12 oldvaluer: DCR_010_R_EN_LG.png  new : DCR-10
index :  13 oldvaluer: 11_hires.png  new : MA-11
index :  14 oldvaluer: SCR_126_R_EN_LG.png  new : SCR-126
index :  15 oldvaluer: SVI_245_R_EN_LG.png  new : SVI-245
index :  16 oldvaluer: LOR_051_R_EN_LG.png  new : LOR-51
index :  17 oldvaluer: EVS_078_R_EN_LG.png  new : EVS-78
index :  18 oldvaluer: XYP_025_R_E

In [ ]:

def scrap_card_data(ptcgo_code,card_number:int,session):

    url = f"https://limitlesstcg.com/cards/{ptcgo_code.upper()}/{card_number}"
    response = session.get(url)
    if response.status_code == 404 : 
        raise ValueError("Page not found, please check urls")

    soup = BeautifulSoup(response.text, 'html.parser')

    # Récupérer le nom du pokemon
    name = soup.find('span', attrs={'class':'card-text-name'}).find('a').contents[0]

    #limitless-card-id & limitless_data_id
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        if " CARD ID " in comment :
            limitless_card_id = comment.replace(" CARD ID ","").strip()
            price_evolution_url = f"https://limitlesstcg.com/api/cards/{limitless_card_id}/prices"
        elif " DATA ID " in comment:
            limitless_data_id = comment.replace(" DATA ID ","").strip()
        else :
            pass
    
    #Prices :
    price_eur = soup.find('span', attrs={'class':'card-price eur'})
    price_eur = float(price_eur.contents[0].replace('€','').replace(",","")) if price_eur != None else None

    price_usd = soup.find('span', attrs={'class':'card-price usd'})
    price_usd = float(price_usd.contents[0].replace('$','').replace(",","")) if price_usd != None else None

    tcgplayer_link = soup.find('a', attrs={'class':'card-buy-button usd'}).attrs['href'] if soup.find('a', attrs={'class':'card-buy-button usd'}) else None
    cardmarket_link = soup.find('a', attrs={'class':'card-buy-button eur'}).attrs['href'] if soup.find('a', attrs={'class':'card-buy-button eur'}) else None
 
    #other cards infos : 
    rarity = soup.find('div', attrs={'class':'prints-current-details'}).find('span',  attrs={'class':None}).contents[0].strip().split(" ")[-1]
    img = soup.find('img', attrs={'class':'card shadow resp-w'}).attrs['data-src']

    #On construit l'objet : 

    result = {
        "id" : ptcgo_code +"-"+ str(card_number),
        "url" : url,
        "set" : ptcgo_code,
        "number" : card_number,
        "name" : name,
        "img" : img,
        "rarity" : rarity,
        "limitless_card_id" : limitless_card_id,
        "limitless_data_id" : limitless_data_id,
        "price_eur" : price_eur,
        "price_usd" : price_usd,
        "price_evolution_url" : price_evolution_url,
        "tcgplayer_link" : tcgplayer_link,
        "cardmarket_link" : cardmarket_link
    }

    return result

def scrap_collection(set_relative_path,session):

    cards_scrapped_data = []
    #Récupération de tous les liens d'une collection
    response = session.get("https://limitlesstcg.com" + set_relative_path)
    collection_page = BeautifulSoup(response.text, 'html.parser')
    # Récupérer la table  <div class="card-search-grid">
    grid = collection_page.find('div', attrs={'class':'card-search-grid'})
    #récupération des adresses des cartes
    for link in grid.find_all('a'):
        infos = link.attrs['href'].split("/")
        card_data = scrap_card_data(ptcgo_code=infos[-2],card_number=infos[-1],session=session)
        cards_scrapped_data.append(card_data)

    return cards_scrapped_data

   

In [ ]:
# Récupérer la liste des collections, avec leurs lien :

# Effectuer la requête HTTP pour obtenir le contenu de la page
response = session.get("https://limitlesstcg.com/cards/")
soup = BeautifulSoup(response.text, 'html.parser')

# Récupérer la table
collections_urls=[]
table = soup.find('table', attrs={'class':'data-table sets-table striped'})

rows = table.find_all('tr')
for row in rows:
    links = row.find_all('a')
    for link in links:
        link = link.get('href')
        if link.startswith('/cards'):
            collections_urls.append(link)

#Processing list (duplicates, etc...)
limitless_sets = list(set(collections_urls))

In [ ]:
limitless_sets = limitless_sets
results = []

for i, card_set in enumerate(limitless_sets):
    try :
        results.extend(scrap_collection(card_set,session))
        print(str(i+1),"/",str(len(limitless_sets)), " - ", card_set, " has been added extracted")
    except Exception as e :
        print(f"Error on set {card_set} ({i}) - ", str(e))

with open('./data/scrap/cards_metadata_v2.pickle', 'w') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [7]:
import json
with open("./data/scrap/metadatav3.json") as f:
   new_metadata = json.load(f)

with open("faiss_db/pokemon-cards_schema_old.json") as f:
   old_metadata = json.load(f)



In [14]:
for item in old_metadata:
    item['url_part']=   item['id'].split("_")[0] +"/"+  "_".join(item['id'].split("_")[1::])

old_metadata    

[{'id': 'AOR_AOR_098_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_098_R_EN_LG.png'},
 {'id': 'AOR_AOR_092_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_092_R_EN_LG.png'},
 {'id': 'AOR_AOR_057_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_057_R_EN_LG.png'},
 {'id': 'AOR_AOR_088_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_088_R_EN_LG.png'},
 {'id': 'AOR_AOR_002_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_002_R_EN_LG.png'},
 {'id': 'AOR_AOR_049_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_049_R_EN_LG.png'},
 {'id': 'AOR_AOR_093_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_093_R_EN_LG.png'},
 {'id': 'AOR_AOR_099_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_099_R_EN_LG.png'},
 {'id': 'AOR_AOR_073_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_073_R_EN_LG.png'},
 {'id': 'AOR_AOR_041_R_EN_LG.png',
  'collection': 'AOR',
  'url_part': 'AOR/AOR_041_R_EN_LG.png'},


In [16]:
#final result : 
final_table = []



for idx,metadata in enumerate(old_metadata):
    for scrapped_data in new_metadata:
        if metadata['url_part'].replace("_LG.",".") in scrapped_data['img']:
            final_table.append({
                "order" : idx,
                "id" : scrapped_data['id'],
                "url" : scrapped_data['url'],
                "set" : scrapped_data['set'],
                "number" : scrapped_data['number'],
                "name" : scrapped_data['name'],
                "img" : scrapped_data['img'],
                "rarity" : scrapped_data['rarity'],
                "limitless_card_id" : scrapped_data['limitless_card_id'],
                "limitless_data_id" : scrapped_data['limitless_data_id'],
                "price_eur" : scrapped_data['price_eur'],
                "price_usd" : scrapped_data['price_usd'],
                "price_evolution_url" : scrapped_data['price_evolution_url'],
                "tcgplayer_link" : scrapped_data['tcgplayer_link'],
                "cardmarket_link" : scrapped_data['cardmarket_link']
            })

import json
with open(f"./data/scrap/metadatav4.json", "w", encoding="utf-8") as f:
    json.dump(final_table, f, indent=2, ensure_ascii=False)    
            

            



In [19]:
10%5

0

In [18]:
11%2

1

In [22]:
4%3

1